In [2]:
!pip install selenium
!pip install beautifulsoup4
!apt-get update
!apt-get install -y chromium-browser
!apt install chromium-chromedriver

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
chromium-browser is already the newest version (1:85.0.4183.83-0ubuntu2.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 19 not up

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import requests
import re
import selenium
import gspread
import time
import random
print("Current selenium version:", selenium.__version__)
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from collections import deque
from urllib.parse import urlsplit
from oauth2client.service_account import ServiceAccountCredentials
import multiprocessing
import difflib
import sys
from google.auth import exceptions
from google.oauth2 import service_account
!ls /usr/lib/chromium-browser/chromedriver

Current selenium version is: 4.14.0
/usr/lib/chromium-browser/chromedriver


In [21]:
def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver

In [42]:
def email1(url):
  try:
    response = requests.get(url)
    time.sleep(4)
    new_emails = set(re.findall(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,4}", response.text, re.I))
    # email_info+= ', '.join(new_emails)
    # emails.update(new_emails)
    return new_emails
  except (requests.exceptions.MissingSchema, requests.exceptions.ConnectionError):
    print('have error')
    return

In [45]:
def get_email(website, link_contact) :
  if type(website) != None:
    email_info = ""
    original_url = link_contact
    # save urls to be scraped
    unscraped = deque([original_url])
    ### to save scraped urls
    scraped = set()
    ### to save fetched emails
    emails = set()
    email_pattern = r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,4}"
    html = driver.page_source
    try:
      while len(unscraped):
          url = unscraped.popleft()
          scraped.add(url)
          parts = urlsplit(url)
          base_url = "{0.scheme}://{0.netloc}".format(parts)
          if '/' in parts.path:
              path = url[:url.rfind('/') + 1]
          else:
              path = url
          print("Crawling URL %s" % url)

          start_time = time.time()
          process = multiprocessing.Process(target=email1,args=(url,))
          process.start()
          process.join(timeout=10)  # Wait at most 10 secs
          if process.is_alive():
              print("Over 10 secs, skip.")
              continue
          else:
              end_time = time.time()
              temp_emails = email1(url)
              email_info+= ', '.join(temp_emails)
              emails.update(temp_emails)

    except Exception as e:
      print('00001')
      email_pattern = r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,4}"
      html = driver.page_source
      emails = re.findall(email_pattern, html)
      email_info += ', '.join(emails)
      return email_info
    isEmpty = (len(emails) == 0) # Check if email is detected
    if isEmpty:
        email_pattern = r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,4}"
        html = driver.page_source
        emails = re.findall(email_pattern, html)
        email_info += ', '.join(emails)
        return email_info
    else:
        return email_info
    driver.delete_all_cookies

In [51]:
def get_url_company(driver,key_word):
  driver.get('https://www.google.com/search?&hl=en')
  driver.find_element(By.NAME, "q").send_keys(key_word + " " + "website" + Keys.ENTER)
  time.sleep(3)
  # soup = BeautifulSoup(driver.page_source, 'html.parser')
  # time.sleep(3)
  # website = soup.find('div', class_="yuRUbf")
  # link = website.a.get('href')
  first_div_element = driver.find_elements(By.XPATH,'//div[@class="yuRUbf"]')
  if first_div_element:
    best_match_link = None
    best_match_ratio = 0
    # Check each <div> element
    print('###########start')
    for div_element in first_div_element:
      span_elements = div_element.find_elements(By.TAG_NAME, 'a')
      if span_elements:
        for span in span_elements:
          link = span.get_attribute('href')
          if 'linked' not in link and 'facebook'not in link and 'wiki'not in link and 'wmb'not in link :
            # Calculate ratio between actual company name vs link
            similarity_ratio = difflib.SequenceMatcher(None, key_word.lower(), link.lower()).ratio()
            if len(link) <= 40 and similarity_ratio > best_match_ratio:
                best_match_link = link
                best_match_ratio = similarity_ratio
    if best_match_link:
      print(key_word)
      print(best_match_link)
    print('###########end')
  return best_match_link

In [50]:
driver = web_driver()
err=''
url_sheet ='https://docs.google.com/spreadsheets/d/1lkQE12IVMfiNJAAuwUj7jMIWamDp-nqYu-9D7waPPmc/edit?usp=sharing'
try :
  json_keyfile = 'authen.json'
  # Access READ and WRITE permission
  scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
  # Authenticize with JSON Service Account Key
  credentials = ServiceAccountCredentials.from_json_keyfile_name(json_keyfile, scope)
except Exception as e:
  err='0'
  print('miss authentication file, please add file')
try :
  gc = gspread.authorize(credentials)
  sh = gc.open_by_url(url_sheet)
  # If it is the first sheet then pass 0 as the parameter, and so on
  worksheet = sh.get_worksheet(0)
  # Read data from file, excluding the header line
  data = worksheet.get_all_values()[1:]
  column_names = worksheet.row_values(1)
  # Find the location of the column named "Company Name"
  Company_column_index = None
  for i, column_name in enumerate(column_names):
      if column_name == "Company Name":
          print(column_name)
          Company_column_index = i + 1
  if Company_column_index is not None:
    # Get at values in Company Name column
    test_column_values = worksheet.col_values(Company_column_index)
    # Remove titles
    test_column_values = test_column_values[1:]
  else:
      print("Could not find 'test' column in sheet.")
      err='1'
      sys.exit()
  temp = 1
  for row in test_column_values:
    print(row)
    temp +=1
    try :
      website_url = get_url_company(driver,row)
      driver.get('https://www.google.com/search?&hl=en')
      driver.find_element(By.NAME, "q").send_keys(row + " " + "contact" + Keys.ENTER)
      time.sleep(3)
      soup = BeautifulSoup(driver.page_source, 'html.parser')
      website = soup.find('div', class_="yuRUbf")
      link = website.a.get('href')
      email_web = get_email(website,link)
      for i in range(3,5):
        if i == 3:
          if website_url:
            worksheet.update_cell(temp, i, website_url)
          else:
            worksheet.update_cell(temp, i, 'empty')
        if i==4:
          if row:
            worksheet.update_cell(temp, i, email_web)
          else:
            worksheet.update_cell(temp, i, 'empty')
    except Exception as e:
      err='2'
      worksheet.update_cell(temp, 5, str(e))
      pass
except Exception as e:
  pass
  print(f"Error occurred : {err}")
driver.quit()

Company Name
FinditParts
###########start
FinditParts
https://www.finditparts.com/
###########end
Crawling URL https://www.finditparts.com/contact#:~:text=Contact%20the%20FinditParts%20Team&text=Give%20us%20a%20call%20at,7am%20EST%20%2D%207pm%20EST%20weekdays.
Error occurred : 2
